In [1]:
import pandas as pd
import re

In [2]:
ls bart-ridership/

date-hour-soo-dest-2016.csv* station_info.csv*
date-hour-soo-dest-2017.csv*


In [3]:
df_2016 = pd.read_csv('bart-ridership/date-hour-soo-dest-2016.csv')
df_2017 = pd.read_csv('bart-ridership/date-hour-soo-dest-2017.csv')
df_station_info = pd.read_csv('bart-ridership/station_info.csv')

In [4]:
df_2016.head()

,Origin,Destination,Throughput,DateTime
0,12TH,12TH,1,2016-01-01 00:00:00
1,12TH,16TH,1,2016-01-01 00:00:00
2,12TH,24TH,4,2016-01-01 00:00:00
3,12TH,ASHB,4,2016-01-01 00:00:00
4,12TH,BALB,2,2016-01-01 00:00:00


In [5]:
df_2016.shape

(9971582, 4)

In [6]:
df_2017.head()

,Origin,Destination,Throughput,DateTime
0,12TH,19TH,1,2017-01-01 00:00:00
1,12TH,24TH,2,2017-01-01 00:00:00
2,12TH,BAYF,1,2017-01-01 00:00:00
3,12TH,CIVC,5,2017-01-01 00:00:00
4,12TH,COLS,2,2017-01-01 00:00:00


In [7]:
df_2017.shape

(3313625, 4)

In [8]:
df = pd.concat([df_2016, df_2017])

In [9]:
df.shape

(13285207, 4)

In [10]:
df.head()

,Origin,Destination,Throughput,DateTime
0,12TH,12TH,1,2016-01-01 00:00:00
1,12TH,16TH,1,2016-01-01 00:00:00
2,12TH,24TH,4,2016-01-01 00:00:00
3,12TH,ASHB,4,2016-01-01 00:00:00
4,12TH,BALB,2,2016-01-01 00:00:00


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13285207 entries, 0 to 3313624
Data columns (total 4 columns):
Origin         object
Destination    object
Throughput     int64
DateTime       object
dtypes: int64(1), object(3)
memory usage: 506.8+ MB


#### df_station cleanup

In [12]:
df_station_info.head()

,Abbreviation,Description,Location,Name
0,12TH,"1245 Broadway, Oakland CA 94612<br />12th St. ...","-122.271450,37.803768,0",12th St. Oakland City Center (12TH)
1,16TH,"2000 Mission Street, San Francisco CA 94110<br...","-122.419694,37.765062,0",16th St. Mission (16TH)
2,19TH,"1900 Broadway, Oakland CA 94612<br />19th Stre...","-122.268602,37.808350,0",19th St. Oakland (19TH)
3,24TH,"2800 Mission Street, San Francisco CA 94110<br...","-122.418143,37.752470,0",24th St. Mission (24TH)
4,ASHB,"3100 Adeline Street, Berkeley CA 94703<br />As...","-122.270062,37.852803,0",Ashby (ASHB)


In [13]:
df_station_info.shape

(46, 4)

In [14]:
df_station_info['Description'] = df_station_info['Description'].str.replace('<br />','.  ')
df_station_info['Description'].iloc[0]

"1245 Broadway, Oakland CA 94612.  12th St. Oakland City Center Station is in the heart of Downtown Oakland, near historic Old Oakland and Oakland's Chinatown."

#### Parse Logitude and Latitude from 'Location'

In [15]:
longitude = []
latitude = []

for i in range(len(df_station_info['Location'].str.split(','))):
    longitude.append(df_station_info['Location'].str.split(',')[i][0])
    latitude.append(df_station_info['Location'].str.split(',')[i][1])

In [16]:
df_station_info['Longitude'] = pd.Series(longitude)
df_station_info['Latitude'] = pd.Series(latitude)
df_station_info.drop('Location', axis =1, inplace = True)

In [17]:
df_station_info.head()

,Abbreviation,Description,Name,Longitude,Latitude
0,12TH,"1245 Broadway, Oakland CA 94612. 12th St. Oak...",12th St. Oakland City Center (12TH),-122.271450,37.803768
1,16TH,"2000 Mission Street, San Francisco CA 94110. ...",16th St. Mission (16TH),-122.419694,37.765062
2,19TH,"1900 Broadway, Oakland CA 94612. 19th Street ...",19th St. Oakland (19TH),-122.268602,37.808350
3,24TH,"2800 Mission Street, San Francisco CA 94110. ...",24th St. Mission (24TH),-122.418143,37.752470
4,ASHB,"3100 Adeline Street, Berkeley CA 94703. Ashby...",Ashby (ASHB),-122.270062,37.852803


In [18]:
df_location = df_station_info.drop(['Description', 'Name'], axis=1)

In [19]:
df_location.head()

,Abbreviation,Longitude,Latitude
0,12TH,-122.271450,37.803768
1,16TH,-122.419694,37.765062
2,19TH,-122.268602,37.808350
3,24TH,-122.418143,37.752470
4,ASHB,-122.270062,37.852803


In [20]:
df_location['Abbreviation'].unique(), df['Origin'].unique()

(array(['12TH', '16TH', '19TH', '24TH', 'ASHB', 'BALB', 'BAYF', 'CAST',
        'CIVC', 'COLS', 'COLM', 'CONC', 'DALY', 'DBRK', 'DELN', 'DUBL',
        'EMBR', 'FRMT', 'FTVL', 'GLEN', 'HAYW', 'LAFY', 'LAKE', 'MCAR',
        'MLBR', 'MONT', 'NBRK', 'NCON', 'OAKL', 'ORIN', 'PHIL', 'PITT',
        'PLZA', 'POWL', 'RICH', 'ROCK', 'SANL', 'SBRN', 'SFIA', 'SHAY',
        'SSAN', 'UCTY', 'WARM', 'WCRK', 'WDUB', 'WOAK'], dtype=object),
 array(['12TH', '16TH', '19TH', '24TH', 'ASHB', 'BALB', 'BAYF', 'CAST',
        'CIVC', 'COLM', 'COLS', 'CONC', 'DALY', 'DBRK', 'DELN', 'DUBL',
        'EMBR', 'FRMT', 'FTVL', 'GLEN', 'HAYW', 'LAFY', 'LAKE', 'MCAR',
        'MLBR', 'MONT', 'NBRK', 'NCON', 'OAKL', 'ORIN', 'PHIL', 'PITT',
        'PLZA', 'POWL', 'RICH', 'ROCK', 'SANL', 'SBRN', 'SFIA', 'SHAY',
        'SSAN', 'UCTY', 'WCRK', 'WDUB', 'WOAK', 'WSPR'], dtype=object))

In [21]:
df['Origin'].replace('WSPR', 'WARM', inplace=True)
df['Destination'].replace('WSPR', 'WARM', inplace=True)

In [22]:
Longitude = df_location.drop('Latitude', axis=1).set_index('Abbreviation').T.to_dict('list')
Latitude = df_location.drop('Longitude', axis=1).set_index('Abbreviation').T.to_dict('list')

In [23]:
df['Longitude_Origin']= df['Origin'].apply(lambda x: Longitude[x][0])
df['Latitude_Origin']= df['Origin'].apply(lambda x: Latitude[x][0])

In [24]:
df['Longitude_Dest']= df['Destination'].apply(lambda x: Longitude[x][0])
df['Latitude_Dest']= df['Destination'].apply(lambda x: Latitude[x][0])

In [ ]:
df.duplicated().sum()

In [25]:
df.head(100)

,Origin,Destination,Throughput,DateTime,Longitude_Origin,Latitude_Origin,Longitude_Dest,Latitude_Dest
0,12TH,12TH,1,2016-01-01 00:00:00,-122.271450,37.803768,-122.271450,37.803768
1,12TH,16TH,1,2016-01-01 00:00:00,-122.271450,37.803768,-122.419694,37.765062
2,12TH,24TH,4,2016-01-01 00:00:00,-122.271450,37.803768,-122.418143,37.752470
3,12TH,ASHB,4,2016-01-01 00:00:00,-122.271450,37.803768,-122.270062,37.852803
4,12TH,BALB,2,2016-01-01 00:00:00,-122.271450,37.803768,-122.447506,37.721585
...,...,...,...,...,...,...,...,...
95,19TH,RICH,1,2016-01-01 00:00:00,-122.268602,37.808350,-122.353099,37.936853
96,19TH,ROCK,4,2016-01-01 00:00:00,-122.268602,37.808350,-122.251371,37.844702
97,19TH,SANL,9,2016-01-01 00:00:00,-122.268602,37.808350,-122.160844,37.721947
98,19TH,SBRN,2,2016-01-01 00:00:00,-122.268602,37.808350,-122.416287,37.637761


In [35]:
df_through = df[df.columns[0:3]]
df_through.head(30)

,Origin,Destination,Throughput
0,12TH,12TH,1
1,12TH,16TH,1
2,12TH,24TH,4
3,12TH,ASHB,4
4,12TH,BALB,2
5,12TH,BAYF,2
6,12TH,CIVC,7
7,12TH,CONC,2
8,12TH,DBRK,6
9,12TH,DELN,2


In [51]:
df_though_sums_origin = pd.DataFrame(df_through.drop('Destination', axis=1).groupby('Origin', as_index=False)['Throughput'].sum())
df_though_sums_dest = pd.DataFrame(df_through.drop('Origin', axis=1).groupby('Destination', as_index=False)['Throughput'].sum())

In [63]:
df_though_sums_origin.rename(columns={'Origin': 'Station','Throughput': 'Origin_Through'}, inplace=True)

In [64]:
df_though_sums_dest.rename(columns={'Destination': 'Station','Throughput': 'Dest_Through'}, inplace=True)

In [68]:
df_though_sums = pd.merge(df_though_sums_origin, df_though_sums_dest, on='Station')
df_though_sums.head()

,Station,Origin_Through,Dest_Through
0,12TH,5322811,5391954
1,16TH,5365000,5384505
2,19TH,5213690,5179220
3,24TH,5459492,5303096
4,ASHB,2297869,2219566


In [69]:
df_though_sums['Total_Through'] = df_though_sums['Origin_Through'] + df_though_sums['Dest_Through']

In [78]:
df_though_sums.head()

,Station,Origin_Through,Dest_Through,Total_Through
0,12TH,5322811,5391954,10714765
1,16TH,5365000,5384505,10749505
2,19TH,5213690,5179220,10392910
3,24TH,5459492,5303096,10762588
4,ASHB,2297869,2219566,4517435


In [84]:
df_though_sums.drop(['Origin_Through', 'Dest_Through'], axis=1, inplace=True)

In [86]:
df_though_sums.sort_values('Total_Through', ascending=False).head(10)

,Station,Total_Through
16,EMBR,34076644
25,MONT,33063895
33,POWL,26593039
8,CIVC,19536703
13,DBRK,10806360
3,24TH,10762588
1,16TH,10749505
0,12TH,10714765
2,19TH,10392910
5,BALB,9070671


#### So, the Station with the most Throughput is 'EMBR' - Embarcadero